# lab_02 - Wprowadzenie do biblioteki Dask.

## 1. Krótki opis biblioteki Dask.

Dask jest zbiorem rozwiązań, który pozwala na zrównoleglenie obliczeń w języku Python oraz przetwarzanie zbiorów większych niż dostępna pamięć RAM. Ekosystem Dask składa się z wielu elementów. 

**Kolekcje Dask (Dask collections):**
* Kolekcje wysokiego poziomu
  * Dask Dataframe
  * Dask Array
  * Dask Bag
* Niskopoziomowe kolekcje
  * Dask Dalayed & Futures

**Klaster Dask**

**Inne elementy ekosystemu Dask:**

* Dask-ML (parallel scikit-learn-style API)
* Dask-image
* Dask-cuDF
* Dask-sql
* Dask-snowflake
* Dask-mongo
* Dask-bigquery


![dask overwiev](dask-overview.svg)


_źródło: dask.org_

**Instalacja Dask**

Oficjalna dokumentacja: https://docs.dask.org/en/stable/install.html

Instalacja podstawowej biblioteki Dask jest bardzo prosta:

```bash
python -m pip install dask
```

Dask posiada jednak duży zbiór opcjonalnych modułów, które mogą się przydać w zależności od potrzeb i zakresu serwisów oraz źródeł danych, z których chcemy skorzystać. Można więc zainstalować również wszystko bez zwracania uwagi na szczegóły:

```bash
python -m pip install dask[complete]
```

Szczegóły instalowanych zależności i ich zastosowanie znajduje się w oficjalnej dokumentacji.

## 2. Dask DataFrame.

> **Oficjalna dokumentacja:** https://docs.dask.org/en/latest/dataframe.html

Ramka Dask jest bardzo zbliżona do ramki pandas w kontekście obsługi z poziomu programisty. Główne różnice są ukryte w sposobie jej przechowywania i wykonywania obliczeń. Obliczenia odbywają się w sposób rozproszony i zrównoleglony.
Dask DataFrame składa się z wielu ramek pandas, odpowiednio podzielonych, aby można było zarówno dane jak i obliczenia wykonać na wielu węzłach (ang. worker) jednocześnie.


![dask_dataframe](dask-dataframe.svg)
_źródło: dask.org_

In [ ]:
# obserwuj komunikaty, aby zainstalować ewentualnie brakujące komponenty
# w komórce wynikowej notebooka pojawi się link do dashboardu, w którym można obserwować aktualne
# zadania i obciążenie klastra jak i zadania wykonane
# UWAGA! Po doinstalowaniu biblioteki bokeh może być konieczne zrestartowanie jądra Pythona i całego notebooka
# aby dashboard poprawnie działał. Każde kolejne uruchomienie poniższego kodu bez restartu jądra, utworzy nową
# instancję klienta

from dask.distributed import Client

client = Client(n_workers=4)
client

Powyższy kod uruchamia instancję lokalnego klastra, który określna 4 workery, między które będzie rozdzielana praca do wykonania. Konfiguracja tego klastra jest tutaj zredukowana do minimum, gdyż bardziej szczegółowo zostanie to omówione na kolejnych zajęciach. Warto tu wiedzieć, że możemy określić czy zadania będą uruchamiane w ramach nowych procesów, czy wątków. Możemy również wskazać ile wątków na worker przypadnie jeżeli na wątki się zdecydujemy. Możemy również określić limit pamięci na worker, co jest dobrym pomysłem gdyż pozostawienie tego parametru z wartością domyślną rozdzieli pamięć po równo na każdy z workerów. Warto pozostawić systemowi hosta trochę zasobów (można sprawdzić wcześniej ile zasobów zużywa system "na jałowym biegu").

Szczegóły API dla lokalnego klastra znajdziemy tu: https://distributed.dask.org/en/latest/api.html#distributed.LocalCluster

W celu pogłębienia wiedzy o niskopoziomowych niuansach działania Pythona, a szczególnie w kontekście współbieżności (lub jej braku) zachęcam do oglądnięcia wystąpienia Marcina Kawki pod tytułem "Wątki i procesy, czyli o zrównoleglaniu programów w Pythonie" na Pytech Summit 2022. Film dostępny pod adresem: https://www.youtube.com/watch?v=kRy_UwUhBpo

Więcej szczegółów oraz film z wprowadzeniem do dashboardu Dask znajdziesz pod adresem: https://docs.dask.org/en/stable/dashboard.html

In [ ]:
client.scheduler_info

In [ ]:
import os
import dask

In [ ]:
import dask.dataframe as dd


# dane, które są przykładowe i nie są dołączone do notebooka
# to dane, które ze zbioru zamowienia.csv zostały sztucznie zwielokrotnione, podzielone i zapisane w kilku
# plikach .csv
ddf = dd.read_csv(os.path.join("..", "lab_01", "data", "*.csv"))

In [ ]:
ddf

Dask nie wykonuje operacji w sposób, do którego możemy być przyzwyczajeni. Tutaj mamy do czynienia z mechnizmem leniwym (ang. lazy), a tym przypadku _lazy loading_, gdzie dask sprawdził ile plików jest do wczytania, podzielił pracę na 20 części oraz na podstawie kilku pierwszych linii z pierwszego pliku ustawił nagłówki kolumn i założył typy danych (które w momencie wystąpienia niespójności w kolumnach mogą ulec zmianie).

In [ ]:
ddf.visualize()

Aby faktycznie wywołać obliczenia musimy wywołać metodę `.compute()` lub jedną z metod, która ją wywołuje niejawnie np. `len`, `head`, `tail`.

In [ ]:
# przed uruchomieniem bieżącej komórki ustaw kartę przeglądarki z dashboardem tak, aby byłą również widoczna
# będzie można śledzić pracę klastra

ddf.head()

In [ ]:
# podobnie jak na zajęciach poprzednich możemy sprawdzić ilość pamięci niezbędnej do przechowania danych ramki
# pamiętajmy o mechanizmie leniwego wywołania, który przygotuje graf obliczeń, ale ich faktycznie jeszcze nie wykona
ddf.memory_usage(deep=True).visualize()

In [ ]:
# ponownie można śledzić zadania w dashboardzie
ddf.memory_usage(deep=True).compute()

In [ ]:
utarg_sum = ddf.groupby(['Kraj']).Utarg.sum()
utarg_sum.visualize()

In [ ]:
%%time
utarg_sum.compute()

Obliczenia odbywają się w sposób rozproszony i w zależności od wielkości zbioru danych oraz ilości workerów, może zająć więcej czasu niż wykonanie obliczeń lokalnie, jeżeli wybrana porcja danych zmieściłaby się pamięci operacyjnej. Możliwe jest wykonanie operacji zapisania dask dataframe w pamięci w celu przyspieszenia obliczeń na mniejszych fragmentach zbioru. Do zapisania danych w pamięci RAM wykorzystujemy metodę `.persist()`.

In [ ]:
polska = ddf[ddf['Kraj'] == 'Polska']

In [ ]:
%%time
# obliczenia w sposób load and select (Dask usuwa z pamięci dane będące obliczeniami pośrednimi z grafu,
# stąd nie może ich ponownie wykorzystać jeżeli obliczenia są takie same)
polska['Utarg'].sum().compute()

In [ ]:
# zapisanie w pamięci RAM
polska = polska.persist()

In [ ]:
%%time
# obliczenia w sposób rozproszony
polska['Utarg'].sum().compute()

## 3. Dask Array.

Tablice Dask są skonstruowane wedle takiej samej idei jak ramki danych. Są to struktury, które wewnątrz składają się z wielu tablic numpy, które są wynikiem podzielenia oryginalj tablicy na mniejsze części, aby zrównoleglić i rozproszyć obliczenia.



![dask array](dask-array.svg)

_źródło: dask.org_

In [ ]:
import numpy as np
import dask.array as da

Tablica numpy oraz obliczenie średniej z tej tablicy wartości próbkowanych z rozkładu normalnego o danych parametrach.

In [ ]:
%%time
arr = np.random.normal(5, 0.2, size=(20_000, 20_000))
arr_mean = arr.mean(axis=0)
arr_mean

Teraz ta sama operacja z wykorzystaniem tablicy Dask.

In [ ]:
# tutaj określimy wielkość chunka ręcznie
darr = da.random.normal(5, 0.2, size=(20_000, 20_000), chunks=(2000, 2000))
darr

In [ ]:
darr_mean = darr.mean(axis=0)
darr_mean

In [ ]:
# uwaga z wywolaniem dla dużej liczby chunków tablicy dask!
darr_mean.visualize()

In [ ]:
%%time
darr_mean.compute()

In [ ]:
# stworzenie tej samej tablicy i przekazanie daskowi zadania automatycznego określenia
# wielkości chunka
darr = da.random.normal(5, 0.2, size=(20_000, 20_000))
darr

Dask przydzielił ilość i wielkość chunków, która bardziej odpowiada architekturze komputerów (system dwójkowy) niż systemowi, w którym człowiek czuje się lepiej (system dziesiętny).
Więcej o tym mechanizmie można doczytać tu: https://docs.dask.org/en/stable/array-chunks.html#automatic-chunking, a kilka sprawdzonych porad co do ich wielkości można również znaleźć tutaj: https://tutorial.dask.org/02_array.html#Rules-of-thumb

In [ ]:
%%time
darr.mean(axis=0).compute()

## **Zadania**

**Zadanie 1**  
Wczytaj plik `zamowienia.csv` do ramki pandas, a następnie w kilku miejscach (ale nie w pierwszych 10 wierszach) wstaw wartość NaN, aby zasymulować wartości brakujące. Zapisz ramkę do pliku `zamowienia_missing.csv`. Wczytaj teraz plik do ramki Dask i sprawdź jakie typy danych zostały przydzielone. Czy zgadzają się z typami z oryginalnego pliku? Wykonaj dowolne obliczenia na całej ramce Dask, aby wymusić wywołanie `.compute()`. Czy pojawił się błąd dotyczący niespójności typów danych? Spróbuj uruchomić kilka razy funkcję wczytywania danych do ramki Dask dataframe z różnymi wartościami parametru `samples`. Dokumentacja `dask.dataframe.read_csv()`: https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html

**Zadanie 2**  
Ze strony https://docs.dask.org/en/stable/dashboard.html skonfiguruj plugin Dask dashboard dla Jupyter Lab i przetestuj jego działanie.

**Zadanie 3**  
Skonfiguruj lokalny klaster (`Client`) tak, aby nie zaalokował wszystkich zasobów (np. zostaw 8 GB RAM dla systemu hosta + 2 rdzenie). Pobierz dane udostępnione na poprzednich zajęciach (https://huggingface.co/datasets/vargr/private_instagram/tree/refs%2Fconvert%2Fparquet/default/train) i załaduj do ramki Dask tyle części ile zdołasz w formie bez optymalizacji. Zmierz czas tej operacji. 

**Zadanie 4**  
Wykonaj kilka operacji na klastrze lokalnym z danymi z zadania 3:
* wyświetl top 10 użytkowników z najwyższą liczbą like'ów,
* pobierz dane tylko za pierwsze półrocze 2019 roku.
Każdorazowo zmierz i wyświetl czas operacji i obserwuj dashboard.

**Zadanie 5**  
Wczytaj te same dane do ramki Dask co w zadaniu 3, ale podaj typy danych, które zostały wybrane w procesie optymalizacji wykonanej w zadaniach z lab 01. Porównaj czas ładowania z zadaniem 3. Wykonaj również te same operacje co w zadaniu 4 i porównaj czas. Śledź wykonanie zadań patrząć na graf wywołań.

**Zadanie 6**  
Podziel tablicę `darr` z przykładów na inne liczby chunków (eksperymentuj) i wykonaj te same obliczenie (średnia). Dla każdej liczby chunków wypisz czas obliczeń (wykonaj to samo obliczenie minimum 10 razy, aby nieco uwiarygodnić wyniki i uśrednij) i porównaj wyniki. Napisz wniosek o wynikach swoich eksperymentów i automatycznego podziału na chunki. Czy udało Ci się osiągnąć lepszą wydajność niż przy domyślnych ustawieniach?